# 3. Allocations

In [171]:
import pandas as pd
import numpy as np
from plotnine import ggplot, aes, geom_bar, labs

file_path = "./data/multi_asset_etf_data.xlsx"
excess_returns_df = pd.read_excel(file_path, 
                                  sheet_name="excess returns", 
                                  parse_dates=["Date"],
                                  index_col="Date") # monthly excess returns

excess_returns_df = excess_returns_df.drop(columns=["QAI"]) #resolve issue with data

In [172]:
MU_TARGET = 0.01
assets = excess_returns_df.columns

def calc_tan_weights(returns):
    """
    Calculate tangency weights for a set of returns
    """
    cov_matrix = returns.cov() 
    cov_vals = cov_matrix.values
    mean_vals = returns.mean().values

    w_unnormalized = np.linalg.inv(cov_vals) @ mean_vals
    w_tan = w_unnormalized / np.sum(w_unnormalized)

    return w_tan


def performance(weights, returns, label):
    """
    Calculate performance of weighted portfolios
    """
    portfolio_returns = returns @ weights

    annualized_mean = portfolio_returns.mean() * 12
    annualized_vol = portfolio_returns.std() * np.sqrt(12)
    annualized_sharpe = (portfolio_returns.mean() / portfolio_returns.std()) * np.sqrt(12)

    portfolio_performance = pd.DataFrame({
        "Portfolio Type": [label],
        "Annualized Mean": [annualized_mean],
        "Annualized Volatility": [annualized_vol],
        "Annualized Sharpe Ratio": [annualized_sharpe]
    })

    return portfolio_performance

# Comparison

In [ ]:
num_assets = len(excess_returns_df.columns)
weights = pd.DataFrame(index=excess_returns_df.columns)
weights['Equal Weights'] = 1/num_assets #Equally Weighted
weights['Risk Parity Weights'] = 1 / (excess_returns_df.var()) #Risk Parity Weighted
weights['Mean-Variance Weights'] = calc_tan_weights(excess_returns_df) #Mean-Variance Weighted

weights_scaled = (weights * MU_TARGET) / (excess_returns_df.mean() @ weights)
weights_scaled = weights_scaled.round(4)

In [177]:
#Compare Portfolios:
eq_performance = performance(weights_scaled["Equal Weights"], excess_returns_df, "Equal Weights")
rp_performance = performance(weights_scaled["Risk Parity Weights"], excess_returns_df, "Risk Parity Weights")
mv_performance = performance(weights_scaled["Mean-Variance Weights"], excess_returns_df, "Mean-Variance Weights")

comparison = pd.concat(
        [eq_performance.round(4),
        rp_performance.round(4),
        mv_performance.round(4)],
    )
comparison


,Portfolio Type,Annualized Mean,Annualized Volatility,Annualized Sharpe Ratio
0,Equal Weights,0.12,0.2693,0.4457
0,Risk Parity Weights,0.12,0.2639,0.4547
0,Mean-Variance Weights,0.12,0.0817,1.4692
